In [1]:
!pip install -q lightning wandb torchvision
!pip install pytorch-lightning
!pip install torchmetrics
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

In [4]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer
import lightning.pytorch as pl
from torchmetrics.functional import accuracy

from lightning.pytorch.callbacks import ModelCheckpoint

wandb.login(key='fd85ae65c5e04fb6fea4d31c1180348532db32d6')
wandb_project= 'EE22M008_Assignment_02_PartA_testing'

wandb: Currently logged in as: ee22m008. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


**CONV5 Architecture**

In [7]:
class Convolutional_Neural_Networks(pl.LightningModule):
  def __init__(self, num_channels, num_conv_blocks, num_filters, filter_size, num_classes, activation_function, pooling_choice, num_fc_layers, dense_neurons, drop_rate, bn, optimizer, lr, wd):
    super(Convolutional_Neural_Networks, self).__init__()
    self.num_channels= num_channels
    self.num_conv_blocks = num_conv_blocks
    self.num_filters = num_filters
    self.filter_size = filter_size
    self.num_classes= num_classes
    self.pooling_layer = pooling_choice
    self.num_fc_layers = num_fc_layers
    self.dense_neurons = dense_neurons
    self.batch_normalization= bn
    self.optimisers= optimizer
    self.learning_rate= lr
    self.weight_decay= wd

    if activation_function=='ReLU':
      self.activation_function= nn.ReLU()
    elif activation_function=='GeLU':
      self.activation_function= nn.GELU()
    elif activation_function== 'SeLU':
      self.activation_function= nn.SELU()
    elif activation_function=='SiLU':
      self.activation_function= nn.SiLU()
    elif activation_function=='Mish':
      self.activation_function= nn.Mish()

    # Define convolutional layers with specified activation and pooling
    self.conv_layers = nn.ModuleList()
    for i in range(self.num_conv_blocks):
      # Determine input channels for the current conv layer
      in_channels = self.num_filters[i-1] if i > 0 else self.num_channels
      conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=self.num_filters[i], kernel_size=self.filter_size)
      self.conv_layers.append(conv_layer)
    self.drop_out_cn= nn.Dropout2d(drop_rate)
    self.drop_out_fc= nn.Dropout(drop_rate)
    self.softmax = nn.LogSoftmax(dim=1)

    self.save_hyperparameters()

  def forward(self, x):
    # Pass input through convolutional layers
    for layer in self.conv_layers:
      x = layer(x)
      if self.batch_normalization:
        x.to(device)
        bn= nn.BatchNorm2d(x.size(1))
        x= bn(x)
      x= self.drop_out_cn(x)
      x = self.activation_function(x)
      if self.pooling_layer=='max':
        pl= nn.MaxPool2d(kernel_size=2, stride=2)
      elif self.pooling_layer=='avg':
        pl= nn.AvgPool2d(kernel_size=2, stride=2)
      x= pl(x)
      x= self.drop_out_cn(x)
    # Flatten the output from the last convolution layers
    x = x.view(x.size(0), -1)

    # Pass through fully connected layers
    for i in range(self.num_fc_layers):
      fc = nn.Linear(x.size(1), self.dense_neurons[i])
      x = fc(x)
      # if self.batch_normalization=='yes':
      #   fc_bn= nn.BatchNorm1d(self.dense_neurons[i])
      #   x= fc_bn(x)
      # x= self.drop_out_fc(x)
      x = self.activation_function(x)
    # Apply softmax activation for classification
    last_layer= nn.Linear(x.size(1), self.num_classes)
    x= last_layer(x)
    x = self.softmax(x)
    return x

  def training_step(self, batch, batch_idx):
    img, label = batch

    logits = self(img)
    loss_function = nn.CrossEntropyLoss()
    loss = loss_function(logits, label)

    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, label, 'multiclass', num_classes=self.num_classes)

    self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
    self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)

    return loss

  def validation_step(self, batch, batch_idx):
    img, label = batch

    logits = self(img)
    loss_function = nn.CrossEntropyLoss()
    loss = loss_function(logits, label)

    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, label, 'multiclass', num_classes=self.num_classes)

    self.log('val_loss', loss, prog_bar=True)
    self.log('val_acc', acc, prog_bar=True)

    return loss

  def test_step(self, batch, batch_idx):
    img, label = batch

    logits = self(img)
    loss_function = nn.CrossEntropyLoss()
    loss = loss_function(logits, label)

    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, label, 'multiclass', num_classes=self.num_classes)

    self.log('test_loss', loss, prog_bar=True)
    self.log('test_acc', acc, ,prog_bar=True)

    return loss

  def configure_optimizers(self):
    if self.optimisers=='adam':
      return torch.optim.Adam(self.parameters(), lr= self.learning_rate, weight_decay= self.weight_decay)
    elif self.optimisers=='nadam':
      return torch.optim.NAdam(self.parameters(), lr= self.learning_rate, weight_decay= self.weight_decay)
    elif self.optimisers=='RMSprop':
      return torch.optim.RMSprop(self.parameters(), lr= self.learning_rate, weight_decay= self.weight_decay)
    elif self.optimisers=='adadelta':
      return torch.optim.Adadelta(self.parameters(), lr= self.learning_rate, weight_decay= self.weight_decay)

**Data Preprocessing**

In [8]:
def Data_Preprocessing(augment_data, train_dir, test_dir, batch_size):
  if augment_data:
    # Define transformations for data augmentation
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Adjust mean and std as needed
    ])
  else:
      train_transform = transforms.Compose([
              transforms.Resize((256, 256)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
          ])


  test_transform = transforms.Compose([
              transforms.Resize((256, 256)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
          ])

  # Define dataset
  train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
  test_dataset= datasets.ImageFolder(root=test_dir, transform=test_transform)

  # Determine sizes for train and validation sets
  total_size = len(train_dataset)
  train_size = int(0.8 * total_size)
  val_size = total_size - train_size

  # Randomly split the dataset into train and validation sets
  train_set, val_set = random_split(train_dataset, [train_size, val_size])

  # Create data loaders for train and validation sets
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader, test_loader

**Testing for the best hyperparameters obtained from above hyperparameter sweeps**

In [9]:
sweep_config = {
  'name': 'Naturalist Classification',
    "metric": {
      "name":"val_acc",
      "goal": "maximize"
  },
  'method': 'grid',
  'parameters': {
        'optimizer': {
            'values': ['adam']
        },

        'filters': {
            'values': ['doubling']
        },
        'filter_size': {
           'values': [3]
        },
        'activation_function':{
            'values': ['ReLU']
        },
        'data_augmentation': {
            'values': [True]
        },
        'pooling':{
            'values': ['max']
        },
        'fc_size':{
            'values': [[128]]           # The number of layer's in the FC can be decided by the length of the dense neurons in the list
        },
        'batch_normalization':{
            'values': [True]
        },
        'droprate':{
            'values':[0]
        },
        'epochs': {
            'values': [10]
        },
        'batch_size': {
            'values': [128]
        },
        'num_layers': {
            'values': [5]
        },
        'learning_rate':{
            'values': [0.01]
        },
        'weight_decay':{
            'values': [0.0005]
        }
    }
}

In [10]:
class ImagePredictionLogger(pl.callbacks.Callback):
    def __init__(self, test_samples, num_samples=30):
        super().__init__()
        self.num_samples = num_samples
        self.test_imgs, self.test_labels = test_samples

    def on_validation_epoch_end(self, trainer, pl_module):
        test_imgs = self.test_imgs
        test_labels = self.test_labels
        # Get model prediction
        logits = pl_module(test_imgs)
        preds = torch.argmax(logits, -1)
        # Log the images as wandb Image
        trainer.logger.experiment.log({
            "Test Samples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                           for x, pred, y in zip(test_imgs[:self.num_samples],
                                                 preds[:self.num_samples],
                                                 test_labels[:self.num_samples])]})


In [11]:
def models():
  wandb.init(project= wandb_project)
  config=wandb.config

  train_dir='/content/drive/MyDrive/Dataset_DL/inaturalist_12K/train'
  test_dir='/content/drive/MyDrive/Dataset_DL/inaturalist_12K/val'
  categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

  num_conv_blocks= config.num_layers
  if config.filters=='same_32':
    num_filters= [32, 32, 32, 32, 32]
  elif config.filters=='same_64':
    num_filters= [64, 64, 64, 64, 64]
  elif config.filters=='doubling':
    num_filters= [16, 32, 64, 128, 256]
  elif config.filters=='halving':
    num_filters= [256, 128, 64, 32, 16]

  filter_size= config.filter_size
  activation_function= config.activation_function
  pooling_choice= config.pooling

  num_fc_layers= len(config.fc_size)
  dense_neurons= config.fc_size
  num_epochs= config.epochs
  drop_rate= config.droprate
  bn= config.batch_normalization
  lr= config.learning_rate
  wd= config.weight_decay
  optimizer= config.optimizer

  num_channels= 3
  num_classes= len(categories)
  augment_data= config.data_augmentation
  wandb.run.name = (
        "bn" + str(bn) + "nf" + str(config.filters)
        + "fs" + str(filter_size) + "do" + str(drop_rate)
        + "opt" + str(optimizer) + "da" + str(augment_data)
        + "af" + str(activation_function) + "fc" + str(num_fc_layers)
        + "pl" + str(pooling_choice)
        + "wd" + str(wd) + "epochs" + str(num_epochs)
    )
  train_loader, val_loader, test_loader= Data_Preprocessing(augment_data, train_dir, test_dir, config.batch_size)

  # Samples required by the ImagePredictionLogger callback to log test image predictions.
  test_samples = next(iter(test_loader))

  model= Convolutional_Neural_Networks(num_channels, num_conv_blocks, num_filters, filter_size, num_classes, activation_function, pooling_choice, num_fc_layers, dense_neurons, drop_rate, bn, optimizer, lr, wd)
  wandb_logger= WandbLogger(project=wandb_project, log_model='all')
  checkpoint_callback = ModelCheckpoint(monitor='val_acc', mode='max')
  trainer = Trainer(max_epochs=num_epochs, logger=wandb_logger, callbacks=[ImagePredictionLogger(test_samples), checkpoint_callback])
  trainer.fit(model, train_loader, val_loader)
  trainer.test(dataloaders=test_loader)

  wandb.finish()

In [ ]:
def sweeper(sweep_config,proj_name):
  sweep_id=wandb.sweep(sweep_config,project=wandb_project)
  wandb.agent(sweep_id,models,project=proj_name, count=1)

sweeper(sweep_config,wandb_project)

Create sweep with ID: vhnhpmxv
Sweep URL: https://wandb.ai/ee22m008/EE22M008_Assignment_02_PartA_testing/sweeps/vhnhpmxv


wandb: Agent Starting Run: umv0zk6u with config:
wandb: 	activation_function: ReLU
wandb: 	batch_normalization: True
wandb: 	batch_size: 128
wandb: 	data_augmentation: True
wandb: 	droprate: 0
wandb: 	epochs: 10
wandb: 	fc_size: [128]
wandb: 	filter_size: 3
wandb: 	filters: doubling
wandb: 	learning_rate: 0.01
wandb: 	num_layers: 5
wandb: 	optimizer: adam
wandb: 	pooling: max
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: 
  | Name                | Type       | Params
---------------------------------------------------
0 | activation_function | ReLU       | 0     
1 | conv_layers         | ModuleList | 392 K 
2 | drop_out_cn         | Dropout2d  | 0    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]